<a href="https://colab.research.google.com/github/Vertex138/DS-Unit-2-Regression-Classification/blob/master/submissions/Colin_Brinkley_Regression_Classification_3_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `RidgeRegression`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `RidgeRegression`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv('../data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
# First, get only the data we need:
dateLow = pd.to_datetime('2019-01-01')
dateEnd = pd.to_datetime('2019-05-01')
df['SALE_DATE']= pd.to_datetime(df['SALE_DATE']) 

dfSub = df[(df['SALE_PRICE'] > 100_000) &
           (df['SALE_PRICE'] < 2_000_000) &
           (df['SALE_DATE'] >= dateLow) &
           (df['SALE_DATE'] < dateEnd) &
           (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS')]

In [0]:
# Alright, let's see what the data looks like
dfSub.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,2019-01-01
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,2019-01-01
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,2019-01-02
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,2019-01-02
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,2019-01-02


In [0]:
# Looks at the unique values of non numeric columns
dfSub.select_dtypes(exclude='number').describe().T.sort_values(by='unique')

,count,unique,top,freq,first,last
BUILDING_CLASS_CATEGORY,3151,1,01 ONE FAMILY DWELLINGS,3151,NaN,NaN
APARTMENT_NUMBER,1,1,RP.,1,NaN,NaN
TAX_CLASS_AT_PRESENT,3151,2,1,3111,NaN,NaN
BOROUGH,3151,5,4,1580,NaN,NaN
NEIGHBORHOOD,3151,7,OTHER,2959,NaN,NaN
BUILDING_CLASS_AT_TIME_OF_SALE,3151,11,A1,1186,NaN,NaN
BUILDING_CLASS_AT_PRESENT,3151,13,A1,1185,NaN,NaN
SALE_DATE,3151,91,2019-01-31 00:00:00,78,2019-01-01 00:00:00,2019-04-30 00:00:00
LAND_SQUARE_FEET,3151,1035,"4,000",289,NaN,NaN
ADDRESS,3151,3135,76-04 265TH STREET,2,NaN,NaN


In [0]:
# Convert LAND_SQUARE_FEET to an integer, being a string makes no sense
dfSub['LAND_SQUARE_FEET'] = pd.to_numeric(dfSub['LAND_SQUARE_FEET'].str.replace(',','').str.replace('#','0'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
#Setting up OneHotEncoder
import category_encoders as ce

In [0]:
# Encode TAX_CLASS_AT_PRESENT
encoder = ce.OneHotEncoder(use_cat_names=True)
dfEncode = encoder.fit_transform(dfSub.TAX_CLASS_AT_PRESENT)
dfSub = pd.concat([dfSub, dfEncode], axis=1, join='outer')

In [0]:
# Encode BOROUGH
encoder = ce.OneHotEncoder(use_cat_names=True)
dfEncode = encoder.fit_transform(dfSub.BOROUGH)
dfSub = pd.concat([dfSub, dfEncode], axis=1, join='outer')

In [0]:
# Encode NEIGHBORHOOD
encoder = ce.OneHotEncoder(use_cat_names=True)
dfEncode = encoder.fit_transform(dfSub.NEIGHBORHOOD)
dfSub = pd.concat([dfSub, dfEncode], axis=1, join='outer')

In [0]:
# Encode BUILDING_CLASS_AT_TIME_OF_SALE
encoder = ce.OneHotEncoder(use_cat_names=True)
dfEncode = encoder.fit_transform(dfSub.BUILDING_CLASS_AT_TIME_OF_SALE)
dfSub = pd.concat([dfSub, dfEncode], axis=1, join='outer')

In [0]:
# Encode BUILDING_CLASS_AT_PRESENT
encoder = ce.OneHotEncoder(use_cat_names=True)
dfEncode = encoder.fit_transform(dfSub.BUILDING_CLASS_AT_PRESENT)
dfSub = pd.concat([dfSub, dfEncode], axis=1, join='outer')

In [0]:
# Drop Encoded columns
dfSub = dfSub.drop(columns=['TAX_CLASS_AT_PRESENT','BOROUGH','NEIGHBORHOOD','BUILDING_CLASS_AT_TIME_OF_SALE','BUILDING_CLASS_AT_PRESENT'])

In [0]:
#Drop NaN columns
dfSub.dropna(axis=1,how='any',inplace=True)
dfSub.dropna(axis=0,how='any',inplace=True)
dfSub.shape

(3151, 47)

In [0]:
dfSub

In [0]:
# Split into train and test sets
dateMid = pd.to_datetime('2019-04-01')
trainDF = dfSub[(dfSub['SALE_DATE'] < dateMid)]
testDF = dfSub[(dfSub['SALE_DATE'] >= dateMid)]

In [0]:
# Define features and target
target = 'SALE_PRICE'
highCard = ['SALE_DATE','ADDRESS']
features = trainDF.columns.drop([target] + highCard)

xTrain = trainDF[features]
yTrain = trainDF[target]
xTest = testDF[features]
yTest = testDF[target]

In [0]:
# Setting up SelectKBest
from sklearn.feature_selection import f_regression, SelectKBest

In [0]:
# SelectKBest
kSel = SelectKBest(score_func=f_regression, k=15)
xTrainSel = kSel.fit_transform(xTrain, yTrain)
xTestSel = kSel.transform(xTest)
xTrainSel.shape, xTestSel.shape

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


((2507, 15), (644, 15))

In [0]:
# Look at the selected Features
allNames = xTrain.columns
selMask = kSel.get_support()
selNames = allNames[selMask]
unsNames = allNames[~selMask]

print('Features selected:')
for _ in selNames:
    print(_)
print('\nFeatures not selected:')
for _ in unsNames:
    print(_)

Features selected:
BLOCK
ZIP_CODE
COMMERCIAL_UNITS
TOTAL_UNITS
LAND_SQUARE_FEET
GROSS_SQUARE_FEET
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_FOREST HILLS
BUILDING_CLASS_AT_TIME_OF_SALE_A5
BUILDING_CLASS_AT_TIME_OF_SALE_A3
BUILDING_CLASS_AT_TIME_OF_SALE_S1
BUILDING_CLASS_AT_PRESENT_A5
BUILDING_CLASS_AT_PRESENT_A3
BUILDING_CLASS_AT_PRESENT_S1

Features not selected:
LOT
RESIDENTIAL_UNITS
YEAR_BUILT
TAX_CLASS_AT_TIME_OF_SALE
TAX_CLASS_AT_PRESENT_1
TAX_CLASS_AT_PRESENT_1D
NEIGHBORHOOD_EAST NEW YORK
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_ASTORIA
BUILDING_CLASS_AT_TIME_OF_SALE_A9
BUILDING_CLASS_AT_TIME_OF_SALE_A1
BUILDING_CLASS_AT_TIME_OF_SALE_A0
BUILDING_CLASS_AT_TIME_OF_SALE_A2
BUILDING_CLASS_AT_TIME_OF_SALE_A4
BUILDING_CLASS_AT_TIME_OF_SALE_A6
BUILDING_CLASS_AT_TIME_OF_SALE_A8
BUILDING_CLASS_AT_TIME_OF_SALE_S0
BUILDING_CLASS_AT_PRESENT_A9
BUILDING_CLASS_AT_PRESENT_A1
BUILDING_CLASS_AT_PRESENT_A0
BUILDING_CLASS_AT_PRESENT_A2
BUILDING_CLASS_A

In [0]:
# Calculate MAE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for _ in range(1, len(xTrain.columns)+1):
    
    print(str(_),'features')
    
    kSel = SelectKBest(score_func=f_regression, k=_)
    xTrainSel = kSel.fit_transform(xTrain, yTrain)
    xTestSel = kSel.transform(xTest)
    
    model = LinearRegression()
    model.fit(xTrainSel, yTrain)
    yPred = model.predict(xTrainSel)
    
    mae = mean_absolute_error(yTest, yPred)
    print('Test MAE: '+str(mae))

1 features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


ValueError: ignored